In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

def get_transition_flux(From, To):
    answer = []
    for i in range(0,len(transition_reduced)):
        if transition_reduced[i][0]==From and transition_reduced[i][1]==To:
            answer.append(flux_reduced[i])
    if len(answer)>1:
        print(f'Multiple fluxes recorded as {From} -> {To} transitions')
        return answer
    elif len(answer)==0:
        if To <3:
            print('Only Balmer and above final energy levels are included in this dataset')
        print(f'No results detected for {From} -> {To} transition')
    else:
        return answer[0]

def get_ratio(From_numerator, To_numerator, From_denominator, To_denominator):
    return get_transition_flux(From_numerator,To_numerator)/get_transition_flux(From_denominator,To_denominator)


def get_flux():
    Continuum = get_continuum_around(Lya_feature,xpixel,ypixel,show_plot = True)[0]
    Normalized_flux = get_continuum_around(Lya_feature,xpixel,ypixel,show_plot=False)[1]
    DataTest = Normalized_flux
    Lya_mask = (WL > 3700*Z1 / 3.226 - Cut_from) & (WL < 4400*Z1 / 3.226 - Cut_from)
    Xrange = np.linspace(3700*Z1 / 3.226 - Cut_from, 4400*Z1 / 3.226 - Cut_from, len(WL[Lya_mask]))
    ZoomMask = ((WL > 3500*Z1 / 3.226-Cut_from) & (WL < 4250*Z1 / 3.226-Cut_from))

    Amp1 = 0.5
    Mean1 = 3926*Z1 / 3.226-Cut_from
    StdDev1 = 1.5

    Amp2 = -4
    Mean2 = 3902*Z1 / 3.226-Cut_from
    FWHM_L = 30
    FWHM_G = 30

    double_func_guess = double_fit_func(Xrange, Amp1, Mean1, StdDev1, Amp2, Mean2, FWHM_L, FWHM_G)

    # Define the bounds for each parameter [amp1_min, mean1_min, stddev1_min, amp2_min, mean2_min, fwhm_L_min, fwhm_G_min],
    #                                      [amp1_max, mean1_max, stddev1_max, amp2_max, mean2_max, fwhm_L_max, fwhm_G_max]
    bounds = ([0, 1210*Z1-Cut_from, 0, -20, 1203*Z1-Cut_from, 0, 0], [20, 1220*Z1-Cut_from, 10, 0, 1216*Z1-Cut_from, 150, 150])


    # Fit the Gaussian + Voigt function to the DataTest array
    params, cov = curve_fit(double_fit_func, Xrange, DataTest[Lya_mask],
                      p0=[Amp1, Mean1, StdDev1, Amp2, Mean2, FWHM_L, FWHM_G],
                      maxfev=20000, sigma = get_weight(Xrange), bounds = bounds)
    print('DataTest shape :' , DataTest.shape)
    # Extract the optimized parameters
    amplitude_fit1, mean_fit1, stddev_fit1, amplitude_fit2, mean_fit2, fwhm_L_fit, fwhm_G_fit= params
    
    # Extract the diagonal elements (variances) from the covariance matrix
    amplitude1_var, mean1_var, stddev1_var, amplitude2_var, mean2_var, fwhm_L_var, fwhm_G_var = np.diag(cov)
    
    # Calculate the uncertainties as square roots of variances
    amplitude1_err, mean1_err, stddev1_err, amplitude2_err, mean2_err, fwhm_L_err, fwhm_G_err = np.sqrt(amplitude1_var), np.sqrt(mean1_var), np.sqrt(stddev1_var), np.sqrt(amplitude2_var), np.sqrt(mean2_var), np.sqrt(fwhm_L_var), np.sqrt(fwhm_G_var)
    
    print("amplitude1:", amplitude_fit1, "+/-", amplitude1_err)
    print("mean1:", mean_fit1 + Cut_from, "+/-", mean1_err)
    print("standard deviation1:", stddev_fit1, "+/-", stddev1_err)
    print('amplitude 2:', amplitude_fit2, "+/-", amplitude2_err )
    print('mean 2:', mean_fit2+Cut_from, "+/-", mean2_err)
    print('FWHM_L:', fwhm_L_fit, "+/-", fwhm_L_err)
    print('FWHM_G:', fwhm_G_fit, "+/-", fwhm_G_err)

    
    #Generate the fitted curve:
    Gaussian_with_Voigt = double_fit_func(Xrange, amplitude_fit1, mean_fit1, stddev_fit1, amplitude_fit2, mean_fit2,
                                      fwhm_L_fit, fwhm_G_fit)
    
    #Normalize the observed and expected frequencies
    #observed_norm = Normalized_flux[Lya_mask]
    #expected_norm = Gaussian_with_Voigt
    #print("obs:", observed_norm.shape)
    #print("exp:", expected_norm.shape)
    #Perform chi-squared analysis
    #chi_square, p_value = chisquare(f_obs=observed_norm,f_exp=expected_norm)
    chi_square = 1
    #if needed, print chi squared stats
    #print("Chi-square:", chi_square)
    #print("p-value:", p_value)
    
    start_tick = WL[Lya_mask][0]
    end_tick = WL[Lya_mask][-1]
    tick_spacing = (len(WL[Lya_mask])/10)
    plt.plot(WL[Lya_mask], Normalized_flux[Lya_mask], linewidth=0.5, label='Normalized_flux Data')
    plt.plot(WL[Lya_mask], Gaussian_with_Voigt, linewidth=0.5, label='fitted curve')
    plt.plot(WL[Lya_mask], double_func_guess, linewidth=0.5, label='guessed')
    plt.axvline(x=3830*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=3856*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=3873*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=3852*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=3975*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=4000*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=4025*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=4045*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=4053*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=4070*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=3897*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    plt.axvline(x=3909*Z1 / 3.226 - Cut_from, linestyle='--',linewidth = 0.5, label='weight boundary')
    tick_positions = np.arange(start_tick, end_tick, tick_spacing)
    tick_labels = [str(int(pos) +Cut_from) for pos in tick_positions]
    plt.xticks(tick_positions, tick_labels)
    plt.xlabel('Wavelength')
    plt.ylabel('Normalized_flux')
    plt.title('Gaussian Fit to Ly-a Feature in pixel')
    #plt.legend(loc='lower left', fontsize=9)
    plt.show()
    difference = Gaussian_with_Voigt - 1
    
    EW_Lya_emit_obs= np.trapz(difference, Xrange)
    EW_Lya_emit_rest_real = (EW_Lya_emit_obs/Z1)
    
    #generate array of values within uncertainties
    amp1_array = random.normal(amplitude_fit1,amplitude1_err,10000)
    stddev1_array = random.normal(stddev_fit1,stddev1_err,10000)
    amp2_array = random.normal(amplitude_fit2,amplitude2_err,10000)
    fwhm_L_array = random.normal(fwhm_L_fit,fwhm_L_err,10000)
    fwhm_G_array = random.normal(fwhm_G_fit,fwhm_G_err,10000)

    
    
    EWs_array = []
    for i in range(0,10000):
        New_double_func = double_fit_func(Xrange, amp1_array[i], mean_fit1, stddev1_array[i], amp2_array[i], mean_fit2,
                                      fwhm_L_array[i], fwhm_G_array[i])
        New_difference = New_double_func-1
        New_EW_obs= np.trapz(New_difference, Xrange)
        New_EW_rest = New_EW_obs/Z1
        EWs_array.append(New_EW_rest)
    print('mean EW: ', np.mean(EWs_array))
    
    EW_uncertainty = np.std(EWs_array)

    print("Rest Frame Equivalent width of Ly-a for {},{}:".format(xpixel, ypixel), EW_Lya_emit_rest_real, "+/-", EW_uncertainty)
 
    return EW_Lya_emit_rest_real, chi_square, EW_uncertainty



def normalize_fluxes_to(transition, fluxes):
    '''normalize all the flux values in 'fluxes' to the given transition's flux value

    -------------

    Parameters
    -------------
    transition :  type = float - right ascension in degrees of query object
    dec :  type = float - declination in degrees of query object
    radius (optional, defaults to 1.2 arcseconds) : 
    retry (optional, defaults to false) : if set to True, if no object is returned 
    
    Returns
    -------------
    original set of fluxes, normalized to given transition
    '''
    
    

print(get_transition_flux(5,3))

print(get_ratio(4,3,5,3))
SED = 'ARM2_HII2_stitch.dat'
raw_file = np.genfromtxt('intrat.out_1e3_8e3', skip_header = 3)
column_names = ['WL (microns)', 'flux density', 'transition (from, to)', 'electron density', 'effective temperature']
WL_reduced = raw_file[:,0]
flux_reduced = raw_file[:,1]
transition_reduced = []
for i in range(0,len(flux_reduced)):
    transition_reduced.append([raw_file[i,3],raw_file[i,5]])
e_density = raw_file[0,7]
Teff = raw_file[0,9]
print(f'example: transition_reduced[1]: {transition_reduced[1]} represents 7->3 transition')

In [ ]:
print(transition_reduced[1])
print(flux_reduced[1])
print(get_transition_flux(4,3))
print(get_transition_flux(5,3))
print(get_ratio(5,3,4,3))
a = ['test']
a.append('string')
print(a)

In [ ]:
transition_reduced

In [ ]:
flux_reduced